In [ ]:
!pip install kfp --upgrade

In [ ]:
!pip install -U google-cloud-pipeline-components

In [1]:
import subprocess
import google_cloud_pipeline_components.aiplatform as gcc_ai

In [2]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics)

from kfp.v2 import compiler

from pipeline.hp_tuning import hyperparameter_tuning_job
from pipeline.preproc_split_dataset import preproc_split_dataset

In [3]:
REGION = 'us-central1'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [ ]:
# ativar antes Cloud Resource Manager API
project_number= subprocess.run(['gcloud', 'projects', 'describe', PROJECT_ID, '--format', 'value(projectNumber)'],
               capture_output=True).stdout.decode('UTF-8').replace('\n','')
sa= '{}-compute@developer.gserviceaccount.com'.format(project_number)
sa

In [ ]:
!gsutil mb -p $PROJECT_ID  gs://kubeflow-lab-v2

In [ ]:
!gsutil ls

In [ ]:
!gsutil iam ch serviceAccount:$sa:objectAdmin gs://kubeflow-lab-v2/ # oferecer ao pipeline permissão para gerir gs

In [4]:
root_path = "gs://kubeflow-lab-v2"
project_id = PROJECT_ID
custom_image = "gcr.io/rugged-shell-320613/hp-tuning:v5"
hp_tuning_name = "lab-hp"

@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=root_path ,
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-test-1")


def pipeline(root_path:str=root_path, project_id:str=project_id,hp_tuning_name:str=hp_tuning_name, custom_image:str=custom_image):
    dataset_op = preproc_split_dataset(root_path=root_path)
    hp_op = hyperparameter_tuning_job(dataset_train=dataset_op.outputs["dataset_train"],
                           dataset_validation=dataset_op.outputs["dataset_validation"],
                                      project=project_id,display_name=hp_tuning_name,image_uri=custom_image)

In [5]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='sklearn_pipe_hp.json')